In [1]:
from cdalvaro.catalogues import OpenClust
from cdalvaro.data_base import DB
from cdalvaro import graphics
from cdalvaro.logging import Logger
from cdalvaro.ml import DEC
from cdalvaro.ml.utils import estimate_n_clusters
from IPython.display import Image
from keras.initializers import VarianceScaling
from keras.optimizers import SGD
from keras.utils import plot_model
import logging
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.manifold import TSNE
from sklearn.preprocessing import MinMaxScaler

Using plaidml.keras.backend backend.
INFO:plaidml:Opening device "metal_amd_radeon_hd_-_firepro_d700.1"


In [2]:
# load dataset
from keras.datasets import mnist
(x, y), (x_test, y_test) = mnist.load_data(path='mnist.npz')
n_clusters = len(np.unique(y))

x = x / 255.0
x = np.reshape(x, [x.shape[0], x.shape[1] * x.shape[2]])

In [3]:
dims = [x.shape[-1], 500, 500, 2000, 10]

loss = 'kld'
optimizer = SGD(1, 0.9)
init = VarianceScaling(scale=1./3., mode='fan_in', distribution='uniform')

# DEC model
dec = DEC(dims=dims, n_clusters=n_clusters, initializer=init)
dec.compile(optimizer=optimizer, loss=loss)
dec.model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           (None, 784)               0         
_________________________________________________________________
encoder_0 (Dense)            (None, 500)               392500    
_________________________________________________________________
encoder_1 (Dense)            (None, 500)               250500    
_________________________________________________________________
encoder_2 (Dense)            (None, 2000)              1002000   
_________________________________________________________________
encoder_3 (Dense)            (None, 10)                20010     
_________________________________________________________________
clustering (ClusteringLayer) (None, 10)                100       
Total params: 1,665,110
Trainable params: 1,665,110
Non-trainable params: 0
_________________________________________________________________


In [4]:
pretrain_epochs = 300
batch_size = 256
maxiter = 2000
update_interval = 140
verbose = 1

dec.pretrain(x, optimizer=optimizer, epochs=pretrain_epochs, batch_size=batch_size)
dec.fit(x, batch_size=batch_size, maxiter=maxiter, update_interval=update_interval, verbose=verbose)

och 93/300
60000/60000 [==============================] - 4s 68us/step - loss: 0.0112
Epoch 94/300
60000/60000 [==============================] - 4s 68us/step - loss: 0.0112
Epoch 95/300
60000/60000 [==============================] - 4s 68us/step - loss: 0.0112
Epoch 96/300
60000/60000 [==============================] - 4s 68us/step - loss: 0.0111
Epoch 97/300
60000/60000 [==============================] - 4s 68us/step - loss: 0.0111
Epoch 98/300
60000/60000 [==============================] - 4s 68us/step - loss: 0.0111
Epoch 99/300
60000/60000 [==============================] - 4s 68us/step - loss: 0.0110
Epoch 100/300
60000/60000 [==============================] - 4s 68us/step - loss: 0.0110
Epoch 101/300
60000/60000 [==============================] - 4s 68us/step - loss: 0.0110
Epoch 102/300
60000/60000 [==============================] - 4s 68us/step - loss: 0.0110
Epoch 103/300
60000/60000 [==============================] - 4s 68us/step - loss: 0.0109
Epoch 104/300
60000/60000 [===

In [5]:
y_pred = dec.predict(x)

In [6]:
relation = dict((x, 0) for x in range(n_clusters))
for i in range(n_clusters):
    u, indeces = np.unique(y_pred[y == i], return_inverse=True)
    i_pred = u[np.argmax(np.bincount(indeces))]
    relation[i_pred] = i
    
relation

{0: 7, 1: 8, 2: 3, 3: 0, 4: 5, 5: 9, 6: 1, 7: 2, 8: 6, 9: 4}

In [7]:
y_pred_corr = np.array(list(map(lambda x: relation[x], y_pred)))

In [8]:
from sklearn.metrics import accuracy_score
accuracy_score(y, y_pred_corr)

0.8172666666666667